## 测试GPU可用性

In [3]:
import tensorrt as trt
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
import torch
# CUDA
print(torch.cuda.is_available())
# cuDNN 深度学习加速
print(torch.backends.cudnn.enabled)
a = torch.rand(5).to('cuda')
a

True
True


tensor([0.6877, 0.4666, 0.2687, 0.0347, 0.5789], device='cuda:0')

# 情感分析


## 默认模型

In [ ]:
import tensorrt as trt
from transformers import pipeline
classifier = pipeline(task="sentiment-analysis")

In [ ]:
title = "Intelligent Systems wins bullish rating from BTIG on Apple Card role"

In [3]:
content = """
BTIG analyst Mark Palmer initiates coverage of small cap Intelligent Systems  INS  0 3   with Buy recommendation for its role in helping Goldman Sachs serve as processor for the launch of the Apple Card 
 We believe INS is an overlooked small cap stock with compelling growth prospects   Palmer writes in a note 
Sees INS s growth to be slower in 2020  though still very strong  as it executes on the Apple Card project   We view its long term growth outlook as attractive and supportive of a premium multiple   he said 
Also sees Apple Card s success making it more likely that Goldman will use INS s CoreCard software for processing projects with other firms in the future 
Quant rating is Bullish
"""

In [5]:
preds = classifier(content)
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
preds

[{'score': 0.636, 'label': 'NEGATIVE'}]

## FinBERT-tone

In [7]:
import tensorrt as trt
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

In [10]:
sentences = ["there is a shortage of capital, and we need extra financing",  
             "growth is strong and we have plenty of liquidity", 
             "there are doubts about our finances", 
             "profits are flat"]
results = nlp(sentences)
print(results) 

[{'label': 'Negative', 'score': 0.9966173768043518}, {'label': 'Positive', 'score': 1.0}, {'label': 'Negative', 'score': 0.9999710321426392}, {'label': 'Neutral', 'score': 0.9889441728591919}]


In [8]:
content = """
BTIG analyst Mark Palmer initiates coverage of small cap Intelligent Systems  INS  0 3   with Buy recommendation for its role in helping Goldman Sachs serve as processor for the launch of the Apple Card 
 We believe INS is an overlooked small cap stock with compelling growth prospects   Palmer writes in a note 
Sees INS s growth to be slower in 2020  th|ough still very strong  as it executes on the Apple Card project   We view its long term growth outlook as attractive and supportive of a premium multiple   he said 
Also sees Apple Card s success making it more likely that Goldman will use INS s CoreCard software for processing projects with other firms in the future 
Quant rating is Bullish
"""
results = nlp(content)
results

[{'label': 'Positive', 'score': 1.0}]